In all versions 5.x the narrative gets cleaned as before and products are cleaned three times. The first time half of the three most occuring entries in __Product__ are randomly removed. The second time the same happens with the six most occuring ones. Finally, the components that almost never occur are removed. This run uses 20 different principle components. 

__Initialing the data__

In [1]:
import pandas as pd

df_selected1 = pd.read_csv("../balanced_data/corpus_balanced_cleaned_lemmatized_both_scarce_elimination.csv", encoding="utf-8")

print("nulls in df_selected:", df_selected1["Consumer complaint narrative"].isnull().sum())
df_selected = df_selected1.dropna()

nulls in df_selected: 5


__Initializing the bag of words__

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

# vect (bag of words)
count_vect = CountVectorizer(
    stop_words="english",
    #ngram_range=(1,2), # bigrammen
    min_df=2, # only keep words that appear twice
    max_df=0.5 # appears max in 50% of documents
)

X_train_counts = count_vect.fit_transform(df_selected["Consumer complaint narrative"])

__Principal components are chosen__

In [3]:
from sklearn.decomposition import TruncatedSVD

tSVD = TruncatedSVD(n_components=20)

principal_components = tSVD.fit_transform(X_train_counts)
print(principal_components.shape)

(203869, 20)


In [ ]:
print(principal_components)
principal_components_df = pd.DataFrame(principal_components)
print(principal_components_df)

__One hot encoding of extra columns__

In [5]:
#One hot encoding
issue_df = pd.Categorical(df_selected['Issue'])

df_dummies = pd.get_dummies(issue_df, prefix = 'issue')

#X_train_counts_df = pd.DataFrame(X_train_counts)
#print("DF conversion done")

print(principal_components_df.shape)
print(df_dummies.shape)
df_concat = pd.concat([principal_components_df, df_dummies], axis = 1)
print(df_concat.shape)


(203869, 20)
(203869, 149)
(203869, 169)


__Split into train and test__

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_concat, df_selected['Product'])
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(152901, 169)
(50968, 169)
(152901,)
(50968,)


__Application of linear SVM__

In [7]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report # do more stats

classifier = svm.LinearSVC()
classifier.fit(X_train, Y_train)

test_predictions = classifier.predict(X_test)
print(classification_report(test_predictions, Y_test))

C:\Users\l-Larsovic-l\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


                                                                              precision    recall  f1-score   support

                                                     Bank account or service       1.00      1.00      1.00      3679
                                                 Checking or savings account       0.99      1.00      1.00      4776
                                                               Consumer Loan       0.96      0.78      0.86      2979
                                                                 Credit card       1.00      1.00      1.00      4594
                                                 Credit card or prepaid card       0.96      0.96      0.96      4655
                                                            Credit reporting       1.00      1.00      1.00      4767
Credit reporting, credit repair services, or other personal consumer reports       0.99      0.88      0.93      6080
                                                       